# Assignment

Deadline: 24 novembre ore 23:59.
Solve efficiently a generic n^2-1 puzzle using path search algorithm.

Cost=  total number of actions you need to __evaluate__. An action is something that bring me to a new state. For example the number of swaps to do.

The result is the sequence of action that took you at the end. The goal is not to find a state but a sequence of actions from srtarting point to end point: we do not look for a soluzion but for a sequence of actions.

# Import and Inizialization

In [394]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappop, heappush
from typing import Tuple, Union

In [395]:
PUZZLE_DIM = 6
action = namedtuple('Action', ['pos1', 'pos2'])

In [396]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

Function to evaluate the quality of a solution (list of actions) as the total number of actions needed.

In [397]:
def qualily(actions):
    return len(actions)

The state is a numpy array.

We created a function that returns the number of actions from a state pos1 to a state pos2.

Compute 100_000 random actions:

In [398]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 161032.31it/s]


array([[32, 25,  0, 24, 30, 15],
       [16, 20, 18,  4, 22, 35],
       [ 3, 29,  8,  2,  9,  1],
       [19, 23, 17,  7,  6, 27],
       [34, 12, 13, 11, 28, 21],
       [33, 10,  5, 26, 14, 31]])

Define a function that indicates if we end the search.

In [399]:
def test_goal(solution):
    arr_solution = np.reshape(solution, PUZZLE_DIM*PUZZLE_DIM)
    arr_solution_no_zero = arr_solution[0: len(arr_solution)-1]
    if np.all(arr_solution_no_zero[:-1] <= arr_solution_no_zero[1:]) and arr_solution[len(arr_solution)-1]==0:
        return True
    return False

# Depth search

Let's use a function that computes a value for each matrix so that it will be easy to store matrices.

This function shouuld be bi-directional: a number is associated to only one matrix and viceversa.

In [400]:
def matrix_score(matrix):
    """
    Performs the product for each element with the position in the matrix.
    
    Parameters:
    - state (np.ndarray): A state of the puzzle.
    
    Returns:
    - Score of the matrix
    """
    #flatter the matrix:
    matrice = matrix.flatten()
    score = 0
    for i in range(len(matrice)):
        score += (10**i)*matrice[i]
    return int(score)


def simplified_matrix_score(matrix):
    """
    Performs the product for each element with the position in the matrix.
    
    Parameters:
    - state (np.ndarray): A state of the puzzle.
    
    Returns:
    - Score of the matrix
    """
    #flatter the matrix:
    matrice = matrix.flatten()
    score = 0
    for i in range(len(matrice)):
        score += i*matrice[i]
    return int(score)

In [401]:
from collections import namedtuple, deque
import numpy as np

# Define action as a named tuple
action = namedtuple('Action', ['pos1', 'pos2'])

def depth_limited_search(initial_state: np.ndarray, final_state: np.ndarray, max_depth: int) -> list[action] or None:
    """
    Performs a depth-limited search to solve the n-puzzle.
    
    Parameters:
    - initial_state (np.ndarray): The starting state of the puzzle.
    - final_state (np.ndarray): The desired goal state of the puzzle.
    - max_depth (int): The maximum depth limit for the search.
    
    Returns:
    - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
    """
    stack = deque([(initial_state, [], 0)])  # Stack of (current state, path to reach it, current depth)
    visited = set()  # Set of visited states for current path only
    optimum = matrix_score(final_state)

    while stack:
        current_state, path, depth = stack.pop()
        current_score = matrix_score(current_state)

        # Check if we reached the goal
        if current_score == optimum:
            return path
        
        # # Backtrack if depth limit reached
        # if depth >= max_depth:
        #     continue
        
        # Add current state to visited set (track only in current path)
        visited.add(current_score)
        
        # Generate and iterate over all possible moves
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            
            # Check if the next state has already been visited in the current path
            if matrix_score(next_state) not in visited:
                # Add the new state and path to stack, increase depth
                stack.append((next_state, path + [act], depth + 1))
        
        # Remove the current state from visited set after backtracking
        #visited.remove(matrix_score(current_state))
    
    return None  # Return None if no solution is found within depth limit

# Iterative Deepening Depth-First Search (IDDFS) wrapper function
def iterative_deepening_dfs(initial_state: np.ndarray, final_state: np.ndarray, max_depth: int = 50) -> list[action] or None:
    """
    Performs an iterative deepening DFS to solve the n-puzzle.
    
    Parameters:
    - initial_state (np.ndarray): The starting state of the puzzle.
    - final_state (np.ndarray): The desired goal state of the puzzle.
    - max_depth (int): The maximum depth to limit the search for IDDFS.
    
    Returns:
    - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
    """
    for depth in range(1, max_depth + 1):
        result = depth_limited_search(initial_state, final_state, depth)
        if result is not None:
            return result
    return None


# Updated Depth search

At each iteration, among all the possible we choose the one with the minimum manahattan distance to the test goal.
In case there is no state with less manhattan distance compared to the current we select the oen that is the "best" among the possibilities. In case they are the same we use the last one.

In [402]:
def update_depth(initial_state: np.ndarray, final_state: np.ndarray) -> list or None:
    stack = deque([(initial_state, [], 0)])  # Stack of (current state, path to reach it, current depth)
    visited = set()  # Set of visited states for current path only
    optimum = matrix_score(final_state)

    while stack:
        current_state, path, depth = stack.pop()
        current_score = matrix_score(current_state)

        # Check if we reached the goal
        if current_score == optimum:
            return path
        
        # Add current state to visited set (track only in current path)
        visited.add(current_score)
        
        # Generate and calculate Manhattan distances for all possible moves
        successors = []
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_score = matrix_score(next_state)
            
            # Check if the next state has already been visited in the current path
            if next_score not in visited:
                # Calculate the Manhattan distance from the goal
                manhattan_dist = manhattan_distance(next_state, final_state)
                # Append (next_state, path + [act], depth + 1, manhattan_dist) to successors list
                successors.append((next_state, path + [act], depth + 1, manhattan_dist))
        
        # Sort successors by Manhattan distance (ascending)
        successors.sort(key=lambda x: x[3])  # Sort by the fourth item, which is manhattan_dist
        
        # Add sorted successors to the stack
        for next_state, new_path, new_depth, _ in successors:
            stack.append((next_state, new_path, new_depth))
        
        # Remove the current state from visited set after backtracking
        #visited.remove(current_score)  # Uncomment if backtracking is implemented
    
    return None  # Return None if no solution is found within depth limit


# A*

In the context of the n-puzzle problem, **Manhattan distance** is a heuristic used to estimate how close a puzzle state is to the goal. For two matrices (representing the current state and the goal state), Manhattan distance is calculated by summing up the individual distances of each tile from its target position.

1. **Tile Distance**: For each tile, the distance is the absolute difference between its current row and target row, plus the absolute difference between its current column and target column.
   
2. **Total Distance**: Summing these values for all tiles gives the Manhattan distance for the puzzle. This value represents the minimum number of moves required to reach the goal state if we could move each tile independently.

In [403]:
def manhattan_distance(state: np.ndarray, goal_state: np.ndarray) -> int:
    """
    Calculates the Manhattan distance for each tile in the puzzle.
    """
    distance = 0
    for value in range(1, state.size):  # Skip 0 (empty space)
        target_pos = np.argwhere(goal_state == value)[0]
        current_pos = np.argwhere(state == value)[0]
        distance += abs(target_pos[0] - current_pos[0]) + abs(target_pos[1] - current_pos[1])
    return distance

In [404]:
def a_star(initial_state: np.ndarray, final_state: np.ndarray) -> list[action] or None:
    """
    Performs an A* search to solve the n-puzzle.
    
    Parameters:
    - initial_state (np.ndarray): The starting state of the puzzle.
    - final_state (np.ndarray): The desired goal state of the puzzle.
    
    Returns:
    - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
    """
    # Priority queue: (f_score, g_score, current_state, path)
    open_set = []
    heappush(open_set, (manhattan_distance(initial_state, final_state), 0, initial_state.tobytes(), []))
    #this function push into open_set a new element continuing to garantee the heap properties.
    # the element that we insert are 4.
    
    visited = set()  
    
    while open_set:
        f_score, g_score, current_bytes, path = heappop(open_set)
        current_state = np.frombuffer(current_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)
        
        # Check if we reached the goal
        if np.array_equal(current_state, final_state):
            return path
        
        # Mark current state as visited
        visited.add(current_bytes)
        
        # Generate and iterate over all possible moves
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_bytes = next_state.tobytes()
            if next_bytes in visited:
                continue
            
            # Calculate scores for the next state
            new_g_score = g_score + 1
            new_f_score = new_g_score + manhattan_distance(next_state, final_state)
            
            # Push the new state into the priority queue with updated scores
            heappush(open_set, (new_f_score, new_g_score, next_bytes, path + [act]))
    
    return None  # Return None if no solution is found

In this code we are storing all the state and comparing them. This may be difficoult. Let's try to use the function that computes a singol number for a given state. This doesn't provide much improvement (find the code in test file).

The problem is that still we are not able to solve 4*4 puzzle.

At first we will try to use a function to be more efficient for storing and comparing the state.

In [405]:
def state_to_tuple(state: np.ndarray) -> tuple:
    return tuple(state.flatten())

We use this function and also change the way we implemented the h function of A*: instead of using only Manhattan distance, we also include other functions, such as linear conflict and we combine the two. 

We may also consider another approach that returns a number representative for the state. This is possible using the hash function over the state:

In [406]:
def state_to_hash(state: np.ndarray) -> int:
    return int("".join(map(str, state.flatten())))

We may also want to try to convert the numpy ndarray into a bytes object directly:

In [407]:
def state_to_bytes(state: np.ndarray) -> bytes:
    return state.tobytes()

In [408]:
class PuzzleHeuristicService:
    def __init__(self, goal_state: np.ndarray):
        self.goal_state = goal_state

    def manhattan_distance(self, state: np.ndarray) -> int:
        size = state.shape[0]
        dist = 0
        for i in range(size):
            for j in range(size):
                value = state[i, j]
                if value != 0:  # Ignore the blanch space
                    target_x, target_y = divmod(np.where(self.goal_state.flatten() == value)[0][0], size)
                    dist += abs(i - target_x) + abs(j - target_y)
        return dist

    def linear_conflict(self, state: np.ndarray) -> int:
        conflict = 0
        size = state.shape[0]
        for row in range(size):
            row_goal = self.goal_state[row]
            row_state = state[row]
            for i in range(size):
                for j in range(i + 1, size):
                    if (
                        row_state[i] in row_goal
                        and row_state[j] in row_goal
                        and row_state[i] > row_state[j]
                    ):
                        conflict += 2
        return conflict

    def combined_heuristic(self, state: np.ndarray) -> int:
        #return self.manhattan_distance(state) + self.linear_conflict(state)
        return self.manhattan_distance(state)




def enhanced_a_star(initial_state: np.ndarray, final_state: np.ndarray) -> Tuple[Union[list, None], float]:
    """
    Enhanced A* algorithm for the n-puzzle problem using modular heuristics.
    """
    heuristic_service = PuzzleHeuristicService(final_state)

    def calculate_heuristic(state: np.ndarray) -> int:
        return heuristic_service.combined_heuristic(state)

    # Priority queue: (f_score, g_score, current_state, path)
    open_set = []
    heappush(open_set, (calculate_heuristic(initial_state), 0, initial_state.tobytes(), []))
    visited = set()
    optimum = state_to_bytes(final_state)

    cost = 0

    while open_set:
        # Extract node with the lowest f score (f score= cost)
        f_score, g_score, current_bytes, path = heappop(open_set)
        current_state = np.frombuffer(current_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)
        current_score = state_to_bytes(current_state)

        # Check if we finished already:
        if current_score == optimum:
            return path, float(cost) 

        # Add current node to visited
        visited.add(current_score)

        # Generate possible moves:
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_score = state_to_bytes(next_state)
            if next_score in visited:
                continue

            cost += 1

            # update scores:
            new_g_score = g_score + 1
            new_f_score = new_g_score + calculate_heuristic(next_state)

            # Add new state to openset
            heappush(open_set, (new_f_score, new_g_score, next_state.tobytes(), path + [act]))

    return None, float('inf')  # No solution found


After some experiments we can say that using only the manhattan distance improves the performance.

# A star with different h function

As heusristics we use 3 different euristcs

In [409]:
class PuzzleHeuristicService:
    def __init__(self, goal_state: np.ndarray):
        self.goal_state = goal_state

    def heuristic_manhattan_distance(self, position):
        distance = 0
        size = len(position)
        for i in range(size):
            for j in range(size):
                tile = position[i][j]
                if tile != 0:
                    target_row = (tile - 1) // size
                    target_col = (tile - 1) % size
                    distance += abs(i - target_row) + abs(j - target_col)
        return distance

    def linear_conflict(self, state: np.ndarray) -> int:
        conflict = 0
        size = state.shape[0]
        for row in range(size):
            row_goal = self.goal_state[row]
            row_state = state[row]
            for i in range(size):
                for j in range(i + 1, size):
                    if (
                        row_state[i] in row_goal
                        and row_state[j] in row_goal
                        and row_state[i] > row_state[j]
                    ):
                        conflict += 2
        return conflict
    

    def heuristic_linear_conflict(self, position):
        conflict = 0
        size = len(position)

        # Row conflicts
        for row in range(size):
            max_val = -1
            for col in range(size):
                value = position[row][col]
                if value != 0 and (value - 1) // size == row:
                    if value > max_val:
                        max_val = value
                    else:
                        conflict += 2

        # Column conflicts
        for col in range(size):
            max_val = -1
            for row in range(size):
                value = position[row][col]
                if value != 0 and (value - 1) % size == col:
                    if value > max_val:
                        max_val = value
                    else:
                        conflict += 2

        return conflict

    def heuristic_walking_distance(self, position):
        # Create a grid to store the walking distances
        size = len(position)
        distance_grid = [[0] * size for _ in range(size)]

        for row in range(size):
            for col in range(size):
                value = position[row][col]
                if value != 0:
                    target_row = (value - 1) // size
                    target_col = (value - 1) % size
                    distance_grid[row][col] = abs(row - target_row) + abs(col - target_col)

        # Calculate the walking distance
        walking_distance = 0
        for row in range(size):
            for col in range(size):
                walking_distance += distance_grid[row][col]

        return walking_distance

    def combined_heuristic(self, state: np.ndarray) -> int:
        #return self.manhattan_distance(state) + self.linear_conflict(state)
        return self.heuristic_manhattan_distance(state) + self.heuristic_linear_conflict(state) + self.heuristic_walking_distance(state)



In [410]:
def enhanced_a_star(initial_state: np.ndarray, final_state: np.ndarray) -> Tuple[Union[list, None], float]:
    """
    Enhanced A* algorithm for the n-puzzle problem using modular heuristics.
    """
    heuristic_service = PuzzleHeuristicService(final_state)

    def calculate_heuristic(state: np.ndarray) -> int:
        return heuristic_service.combined_heuristic(state)

    # Priority queue: (f_score, g_score, current_state, path)
    open_set = []
    heappush(open_set, (calculate_heuristic(initial_state), 0, initial_state.tobytes(), []))
    visited = set()
    optimum = state_to_bytes(final_state)

    cost = 0

    while open_set:
        # Extract node with the lowest f score (f score= cost)
        f_score, g_score, current_bytes, path = heappop(open_set)
        current_state = np.frombuffer(current_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)
        current_score = state_to_bytes(current_state)

        # Check if we finished already:
        if current_score == optimum:
            return path, float(cost) 

        # Add current node to visited
        visited.add(current_score)

        # Generate possible moves:
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_score = state_to_bytes(next_state)
            if next_score in visited:
                continue

            cost += 1

            # update scores:
            new_g_score = g_score + 1
            new_f_score = new_g_score + calculate_heuristic(next_state)

            # Add new state to openset
            heappush(open_set, (new_f_score, new_g_score, next_state.tobytes(), path + [act]))

    return None, float('inf')  # No solution found


# Main

In [411]:
test_goal = np.arange(1, PUZZLE_DIM*PUZZLE_DIM, 1)
test_goal = np.append(test_goal, 0)
test_goal = test_goal.reshape((PUZZLE_DIM, PUZZLE_DIM))

# Try the upgrade depth search

In [412]:
#update_depth(state, test_goal)

# Try the A* algorithm

In [413]:
#a_star(state, test_goal)

before 6 minutes

In [ ]:
actions, costValue = enhanced_a_star(state, test_goal)

In [ ]:
qualily(actions)

122

In [ ]:
actions

[Action(pos1=(2, 2), pos2=(2, 1)),
 Action(pos1=(2, 1), pos2=(2, 0)),
 Action(pos1=(2, 0), pos2=(1, 0)),
 Action(pos1=(1, 0), pos2=(0, 0)),
 Action(pos1=(0, 0), pos2=(0, 1)),
 Action(pos1=(0, 1), pos2=(1, 1)),
 Action(pos1=(1, 1), pos2=(1, 2)),
 Action(pos1=(1, 2), pos2=(0, 2)),
 Action(pos1=(0, 2), pos2=(0, 3)),
 Action(pos1=(0, 3), pos2=(1, 3)),
 Action(pos1=(1, 3), pos2=(2, 3)),
 Action(pos1=(2, 3), pos2=(2, 2)),
 Action(pos1=(2, 2), pos2=(2, 1)),
 Action(pos1=(2, 1), pos2=(1, 1)),
 Action(pos1=(1, 1), pos2=(1, 0)),
 Action(pos1=(1, 0), pos2=(2, 0)),
 Action(pos1=(2, 0), pos2=(2, 1)),
 Action(pos1=(2, 1), pos2=(2, 2)),
 Action(pos1=(2, 2), pos2=(3, 2)),
 Action(pos1=(3, 2), pos2=(4, 2)),
 Action(pos1=(4, 2), pos2=(4, 1)),
 Action(pos1=(4, 1), pos2=(3, 1)),
 Action(pos1=(3, 1), pos2=(3, 2)),
 Action(pos1=(3, 2), pos2=(3, 3)),
 Action(pos1=(3, 3), pos2=(4, 3)),
 Action(pos1=(4, 3), pos2=(4, 2)),
 Action(pos1=(4, 2), pos2=(3, 2)),
 Action(pos1=(3, 2), pos2=(2, 2)),
 Action(pos1=(2, 2),

In [ ]:
state

array([[12, 11, 13,  3, 16],
       [ 4,  2, 21,  9,  8],
       [14, 15,  0,  7, 22],
       [17, 23, 18, 24, 20],
       [ 6, 10,  1,  5, 19]])

In [ ]:
test_goal

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24,  0]])